#  Filling RAG Outputs For Evaluation

In this notebook, we use the deployed RAG pipeline to populate the RAG outputs: contexts (retrieved relevant documents) and answer (generated by RAG pipeline).

The RAG pipeline must be deployed before running this notebook because this notebook communicates with the Chain Server API.
Refer to an example like the basic LangChain example in the `RAG/examples/basic_rag/langchain/` directory, the basic LlamaIndex example in the `RAG/examples/basic_rag/llamaindex/` directory, or any other example to build and start the containers.

To learn more about how the pipeline works in the backend, refer to the `RAG/notebooks/llamaindex/llamaindex_basic_RAG.ipynb` notebook.

### Step 1: Set up Dataset Directory and Define API Endpoints

In [ ]:
!pip install -r requirements.txt

In [ ]:
%%capture
!test -d dataset || unzip ../dataset.zip

In [ ]:
import os
# Give the IP of the machine where the above RAG pipeline is deployed, in place of chain-server.
url_upload = f"http://chain-server:8081/documents"
url_generate = f"http://chain-server:8081/generate"
url_doc_search = f"http://chain-server:8081/search"

### Step 2: Ingest Documents

Ingest the dataset using the /documents endpoint of the Chain Server.

In [ ]:
import os
import requests
import mimetypes

def upload_document(file_path, url):
    headers = {
        'accept': 'application/json'
    }
    mime_type, _ = mimetypes.guess_type(file_path)
    files = {
        'file': (file_path, open(file_path, 'rb'), mime_type)
    }
    response = requests.post(url, headers=headers, files=files)

    return response.text

def upload_pdf_files(folder_path, upload_url):
    for files in os.listdir(folder_path):
        _, ext = os.path.splitext(files)
        # Ingest only pdf files
        if ext.lower() == ".pdf":
            file_path = os.path.join(folder_path, files)
            print(upload_document(file_path, upload_url))

In [ ]:
import time

start_time = time.time()
upload_pdf_files("dataset",url_upload )
print(f"--- {time.time() - start_time} seconds ---")

### Step 3: Fill the RAG Outputs 

Let's now query the RAG pipeline and fill the outputs `contexts` and `answer` on the evaluation JSON file.

First, we need to load the previously generated dataset. So far, the RAG outputs fields are empty.


In [ ]:
# import the relevant libraries
import json
from IPython.display import JSON

# load the evaluation data
f = open('qa_generation.json')
data = json.load(f)

# show the first element
JSON(data[0])

Query the RAG pipeline and populate the `contexts` and `answer` fields.

In [ ]:
import typing

generate_api_params={"use_knowledge_base": True, "temperature":0.2,"top_p":0.7,"max_tokens": 256}
document_search_api_params={"num_docs": 1}
new_data=[]

In [ ]:
for entry in data:
    entry_generate = {
        "messages":[
            {
                "role":"user",
                "content":entry["question"]
            }
        ],
        "use_knowledge_base": generate_api_params["use_knowledge_base"],
        "temperature": generate_api_params["temperature"],
        "top_p": generate_api_params["top_p"],
        "max_tokens": generate_api_params["max_tokens"],
        "stop":[
                "string"
        ]
    }
    entry["answer"] = ""
    try:
        with requests.post(url_generate, stream=True, json=entry_generate) as r:
            for chunk in r.iter_lines():
                raw_resp = chunk.decode("UTF-8")
                if not raw_resp:
                    continue
                resp_dict = None
                try:
                    print(raw_resp)
                    resp_dict = json.loads(raw_resp[6:])
                    resp_choices = resp_dict.get("choices", [])
                    if len(resp_choices):
                        resp_str = resp_choices[0].get("message", {}).get("content", "")
                        entry["answer"] += resp_str
                except Exception as e:
                    print(f"Exception Occured: {e}")
    except Exception as e:
        print(f"Exception Occured: {e}")
        entry["answer"] = "Answer couldn't be generated."
    print(entry["answer"])
    entry_doc_search = {
            "query": entry["question"],
            "top_k": document_search_api_params["num_docs"]
        }
    response = requests.post(url_doc_search, json=entry_doc_search).json()
    context_list =typing.cast(typing.List[typing.Dict[str, typing.Union[str, float]]], response)
    contexts = [context.get("content") for context in context_list['chunks']]
    try:
        entry["contexts"] = [contexts[0]]
    except Exception as e:
        print(f"Exception Occured: {e}")
        entry["contexts"] = ""
    new_data.append(entry)
    print(len(entry["contexts"]))

In [ ]:
# json_list_string=json.dumps(data)

# show again the first element
JSON(data[0])

Save the new evaluation datasets.

In [ ]:
import json
with open('eval.json', 'w') as f:
    json.dump(data, f)